In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
cf.go_offline()
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
import os
from plotly.subplots import make_subplots
from plotly import tools 
import plotly.graph_objects as go
from scipy import stats
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt


In [2]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
train.shape

(1460, 81)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [6]:
null_counts = train.isnull().sum()
null_counts[null_counts > 0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [7]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [8]:
train._get_numeric_data().columns

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [9]:
train._get_numeric_data().iplot(kind="histogram", bins=50)

In [10]:
train.corr().iplot(kind='heatmap',colorscale="Blues", title="Feature Correlation Matrix")

In [11]:
bp = train[['YearBuilt', 'SalePrice']]
bp.pivot(columns='YearBuilt', values='SalePrice').iplot(kind='box')

In [12]:
train.iplot(asFigure=True, x='KitchenQual', y='SalePrice', mode='markers')

In [13]:
train['1stFlrSF'].describe()

count    1460.000000
mean     1162.626712
std       386.587738
min       334.000000
25%       882.000000
50%      1087.000000
75%      1391.250000
max      4692.000000
Name: 1stFlrSF, dtype: float64

In [14]:
train['SaleType'].iplot(kind="histogram", bins=20, theme="white", title="SaleType",xTitle='Sale Type', yTitle='Count')

In [15]:
bp2 = train[['Street', 'SalePrice']]
bp2.pivot(columns='Street', values='SalePrice').iplot(kind='box')

In [16]:
def mostCommon(columnName):
    if train[columnName].value_counts().index[0] == 'None':
        return train[columnName].value_counts().index[1]
    else:
        return train[columnName].value_counts().index[0]

In [17]:
train['Alley'] = train['Alley'].fillna(mostCommon('Alley'))
train['Electrical'] = train['Electrical'].fillna(mostCommon('Electrical'))
train['MasVnrType'] = train['MasVnrType'].replace(to_replace='None', value=mostCommon('MasVnrType'))
train['LotFrontage']= train.LotFrontage.fillna(train.LotFrontage.mean())
train['BsmtQual'] = train['BsmtQual'].fillna(mostCommon('BsmtQual'))
train['MasVnrType'] = train['MasVnrType'].fillna(mostCommon('MasVnrType'))
train['MasVnrArea']= train.MasVnrArea.fillna(train.MasVnrArea.mean())
train['BsmtCond'] = train['BsmtCond'].fillna(mostCommon('BsmtCond'))
train['BsmtExposure'] = train['BsmtExposure'].fillna(mostCommon('BsmtExposure'))
train['BsmtFinType1'] = train['BsmtFinType1'].fillna(mostCommon('BsmtFinType1'))
train['BsmtFinType2'] = train['BsmtFinType2'].fillna(mostCommon('BsmtFinType2'))
train['Electrical'] = train['Electrical'].fillna(mostCommon('Electrical'))
train['FireplaceQu'] = train['FireplaceQu'].fillna(mostCommon('FireplaceQu'))
train['GarageType'] = train['GarageType'].fillna(mostCommon('GarageType'))
train['GarageYrBlt'] = train['GarageYrBlt'].fillna(mostCommon('GarageYrBlt'))
train['GarageFinish'] = train['GarageFinish'].fillna(mostCommon('GarageFinish'))
train['GarageQual'] = train['GarageQual'].fillna(mostCommon('GarageQual'))
train['GarageCond'] = train['GarageCond'].fillna(mostCommon('GarageCond'))
train['PoolQC'] = train['PoolQC'].fillna(mostCommon('PoolQC'))
train['Fence'] = train['Fence'].fillna(mostCommon('Fence'))
train['MiscFeature'] = train['MiscFeature'].fillna(mostCommon('MiscFeature'))
train = train.drop(['Utilities'], axis=1)

In [18]:
fig = make_subplots(rows=2, cols=2)
fig.add_trace(
    go.Scatter(x=list(train['GrLivArea']), y=list(train['SalePrice']),mode="markers+text"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=list(train['TotalBsmtSF']), y=list(train['SalePrice']) ,mode="markers+text"),
    row=1, col=2 
)
fig.add_trace(
    go.Scatter(x=list(train['1stFlrSF']), y=list(train['SalePrice']),mode="markers+text"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=list(train['LotArea']), y=list(train['SalePrice']) ,mode="markers+text"),
    row=2, col=2 
)

fig.update_layout(height=600, width=800, title_text="Before Removing Outliers")
fig.show()

In [19]:
train.shape

(1460, 80)

In [20]:
train = train[(np.abs(stats.zscore(train.select_dtypes(exclude='object'))) < 3).all(axis=1)]

In [21]:
fig = make_subplots(rows=2, cols=2)
fig.add_trace(
    go.Scatter(x=list(train['GrLivArea']), y=list(train['SalePrice']),mode="markers+text"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=list(train['TotalBsmtSF']), y=list(train['SalePrice']) ,mode="markers+text"),
    row=1, col=2 
)
fig.add_trace(
    go.Scatter(x=list(train['1stFlrSF']), y=list(train['SalePrice']),mode="markers+text"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=list(train['LotArea']), y=list(train['SalePrice']) ,mode="markers+text"),
    row=2, col=2 
)

fig.update_layout(height=600, width=800, title_text="After Removing Outliers")
fig.show()

In [22]:
train.shape

(1015, 80)

In [23]:
cor_train = train.select_dtypes(include=[np.number])
cor_train = cor_train.drop('Id', 1)

In [24]:
cor_train.shape

(1015, 37)

In [25]:
corrDf = cor_train.corr()

In [26]:
corrDf.corr().iplot(kind='heatmap',colorscale="Blues", title="Feature Correlation Matrix")

In [27]:
top_features = corrDf.index[abs(corrDf['SalePrice']>0.5)]
top_corr = train[top_features].corr()
top_corr.corr().iplot(kind='heatmap',colorscale="Blues", title="Feature Correlation Matrix")

In [28]:
train_corr = train.corr()
train_corr_sorted = train_corr.sort_values(['SalePrice'], ascending=False)

In [29]:
corrSp = pd.DataFrame()
corrSp['Label'] = train_corr_sorted.SalePrice.index
corrSp['Corr'] = train_corr_sorted.SalePrice.values

In [30]:
fig = px.pie(corrSp, values='Corr', names='Label', title='Sale Price Correlation chart')
fig.show()

In [31]:
categorical_list = []
for i in train.columns.tolist():
    if train[i].dtype=='object':
        categorical_list.append(i)
labelCols = categorical_list + ['MSSubClass', 'MasVnrArea', 'MoSold', 'OverallCond', 'YrSold']

In [32]:
for c in labelCols:
    lbl = LabelEncoder() 
    lbl.fit(list(train[c].values)) 
    train[c] = lbl.transform(list(train[c].values))

In [33]:
salePrice = train['SalePrice']
train = train.drop(['SalePrice'], axis=1)
X = train.values
y = salePrice.values

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [35]:
GBR = GradientBoostingRegressor(n_estimators=100, max_depth=4)

In [36]:
GBR.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [37]:
print("Accuracy: ", GBR.score(X_test, y_test)*100)

Accuracy:  87.72439824903658


In [38]:
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [39]:
test['Alley'] = test['Alley'].fillna(mostCommon('Alley'))
test['Electrical'] = test['Electrical'].fillna(mostCommon('Electrical'))
test['MasVnrType'] = test['MasVnrType'].replace(to_replace='None', value=mostCommon('MasVnrType'))
test['LotFrontage']= test.LotFrontage.fillna(test.LotFrontage.mean())
test['BsmtQual'] = test['BsmtQual'].fillna(mostCommon('BsmtQual'))
test['MasVnrType'] = test['MasVnrType'].fillna(mostCommon('MasVnrType'))
test['MasVnrArea']= test.MasVnrArea.fillna(test.MasVnrArea.mean())
test['BsmtCond'] = test['BsmtCond'].fillna(mostCommon('BsmtCond'))
test['BsmtExposure'] = test['BsmtExposure'].fillna(mostCommon('BsmtExposure'))
test['BsmtFinType1'] = test['BsmtFinType1'].fillna(mostCommon('BsmtFinType1'))
test['BsmtFinType2'] = test['BsmtFinType2'].fillna(mostCommon('BsmtFinType2'))
test['Electrical'] = test['Electrical'].fillna(mostCommon('Electrical'))
test['FireplaceQu'] = test['FireplaceQu'].fillna(mostCommon('FireplaceQu'))
test['GarageType'] = test['GarageType'].fillna(mostCommon('GarageType'))
test['GarageYrBlt'] = test['GarageYrBlt'].fillna(mostCommon('GarageYrBlt'))
test['GarageFinish'] = test['GarageFinish'].fillna(mostCommon('GarageFinish'))
test['GarageQual'] = test['GarageQual'].fillna(mostCommon('GarageQual'))
test['GarageCond'] = test['GarageCond'].fillna(mostCommon('GarageCond'))
test['PoolQC'] = test['PoolQC'].fillna(mostCommon('PoolQC'))
test['Fence'] = test['Fence'].fillna(mostCommon('Fence'))
test['MiscFeature'] = test['MiscFeature'].fillna(mostCommon('MiscFeature'))

test['BsmtFinSF1']= test.BsmtFinSF1.fillna(test.BsmtFinSF1.mean())
test['BsmtFinSF2']= test.BsmtFinSF2.fillna(test.BsmtFinSF2.mean())
test['BsmtUnfSF']= test.BsmtUnfSF.fillna(test.BsmtUnfSF.mean())
test['TotalBsmtSF']= test.TotalBsmtSF.fillna(test.TotalBsmtSF.mean())
test['BsmtFullBath'] = test['BsmtFullBath'].fillna(mostCommon('BsmtFullBath'))
test['BsmtHalfBath'] = test['BsmtHalfBath'].fillna(mostCommon('BsmtHalfBath'))
test['GarageCars'] = test['GarageCars'].fillna(mostCommon('GarageCars'))
test['GarageArea']= test.GarageArea.fillna(test.GarageArea.mean())

test = test.drop(['Utilities'], axis=1)

In [40]:
for c in labelCols:
    lbl = LabelEncoder() 
    lbl.fit(list(test[c].values)) 
    test[c] = lbl.transform(list(test[c].values))

In [41]:
submissionPredicts  = GBR.predict(test)

In [42]:
submissionPredicts

array([129661.34388877, 145941.63608426, 182746.22064952, ...,
       149114.95688398, 101096.04044024, 239184.34317571])

In [43]:
submission = pd.DataFrame()
submission['Id'] = test['Id']
submission['SalePrice'] = submissionPredicts
submission.to_csv('submission.csv',index=False)